In [ ]:
import json
import numpy
import operator

In [ ]:
gt_path = '../../../data/posted_GT/gt_answer_key/'
# New
agg_gt_answ_file = '../../../data/aggregate_gt/agg_answer_key.json'
agg_metric_file = '../../../data/aggregate_gt/agg_metric_parameters.json'
sub_path = '../../../data/team_submissions/ISI/GroundTruth/'

In [ ]:
def clean_sub(sub, index_of_ad, index_of_score):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    for entry in sub:
        # skip over aggregate single answer
        if type(entry) == list:
            if entry[index_of_score] > i:
                print "ORDERING PROBLEM"
                return None
            i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        # skip over aggregate single answer
        if type(entry) == list:
            if entry[index_of_ad] not in seen:
                uniqs.append(entry)
                seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

In [ ]:
def AvgPrecision(gt, sub, index_of_ad, index_of_score):
    score = 100
    seen = []
    rel_docs = len(gt)
    num_hits = 0
    num_retrieved = 0
    cumpre = 0
    for entry in sub:
        num_retrieved += 1
        ad_id = entry[index_of_ad]
        # excessive checking of dupes
        if ad_id in seen:
            print "DUPLICATE"
            return None
        seen.append(ad_id)
        # excessive checking of order
        if entry[index_of_score] > score:
            print "ORDER PROBLEM"
            return None
        score = entry[index_of_score]
        if ad_id in gt:
            num_hits += 1
            cumpre += num_hits/float(num_retrieved)
            
    return cumpre/rel_docs

In [ ]:
def AggMetric(gt_answ, sub_answ, feature):
    scale_cutoff = agg_metric_params[feature]
    diff = gt_answ - sub_answ
    if numpy.absolute(diff) <= (gt_answ * scale_cutoff):
        metric = 1
    else:
        metric = 0
        
    return metric

# SET EXTRACTIONS

In [ ]:
extractions = 'ISI_ISI_only'
#extractions = 'ISI_lattice_only'
#extractions = 'ISI_ISI_and_lattice'

if extractions == 'ISI_ISI_only':
    sub_pf = sub_path + 'resubmission-isi-ground-truth-2016-11-30/post_point_fact_parsed_fixed_all_answers.json'
    sub_id = sub_path + 'resubmission-isi-ground-truth-2016-11-30/post_cluster_identification-parsed_fixed_all_answers.json'
    sub_facet = sub_path + 'resubmission-isi-ground-truth-2016-11-30/post_cluster_facet_parsed_fixed_all_answers.json'
    sub_agg = sub_path + 'resubmission-isi-ground-truth-2016-11-30/post_aggregate_parsed_fixed_all_answers.json'
    
elif extractions == 'ISI_lattice_only':
    sub_pf = sub_path + 'isi-submission-only-lattice-extractions/post_point_fact_parsed_fixed_all_answers.json'
    sub_id = sub_path + 'isi-submission-only-lattice-extractions/post_cluster_identification-parsed_fixed_all_answers.json'
    sub_facet = sub_path + 'isi-submission-only-lattice-extractions/post_cluster_facet_parsed_fixed_all_answers.json'
    sub_agg = sub_path + 'isi-submission-only-lattice-extractions/post_aggregate_parsed_fixed_all_answers.json'
    
elif extractions == 'ISI_ISI_and_lattice':
    sub_pf = sub_path + 'resubmission-dig-plus-lattice-extractions/post_point_fact_parsed_fixed_all_answers.json'
    sub_id = sub_path + 'resubmission-dig-plus-lattice-extractions/post_cluster_identification-parsed_fixed_all_answers.json'
    sub_facet = sub_path + 'resubmission-dig-plus-lattice-extractions/post_cluster_facet_parsed_fixed_all_answers.json'
    sub_agg = sub_path + 'resubmission-dig-plus-lattice-extractions/post_aggregate_parsed_fixed_all_answers.json'
    

## Load Submissions

In [ ]:
f = open(sub_pf, 'r')
pf_sub = eval(f.read())
f.close()

f = open(sub_id, 'r')
id_sub = eval(f.read())
f.close()

f = open(sub_facet, 'r')
facet_sub = eval(f.read())
f.close()

f = open(sub_agg, 'r')
agg_sub = eval(f.read())
f.close()

Submissions = {}
Submissions['Point Fact'] = pf_sub
Submissions['Cluster Identification'] = id_sub
Submissions['Cluster Facet'] = facet_sub
Submissions['Cluster Aggregate'] = agg_sub

del pf_sub
del id_sub
del facet_sub
del agg_sub

## Load GT

In [ ]:
pf_path = gt_path + 'pf_quest2ads.json'
f = open(pf_path, 'r')
pf_gt = eval(f.read())
f.close()

id_path = gt_path + 'id_quest2ads.json'
f = open(id_path, 'r')
id_gt = eval(f.read())
f.close()

facet_path = gt_path + 'facet_quest2ads.json'
f = open(facet_path, 'r')
facet_gt = eval(f.read())
f.close()

agg_path = gt_path + 'agg_quest2ads.json'
f = open(agg_path, 'r')
agg_gt = eval(f.read())
f.close()

# New
f = open(agg_gt_answ_file, 'r')
agg_answ_gt = eval(f.read())
f.close()

f = open(agg_metric_file, 'r')
agg_metric_params = eval(f.read())
f.close()

Ground_Truth = {}
Ground_Truth['Point Fact'] = pf_gt
Ground_Truth['Cluster Identification'] = id_gt
Ground_Truth['Cluster Facet'] = facet_gt
Ground_Truth['Cluster Aggregate'] = agg_gt

del pf_gt
del id_gt
del facet_gt
del agg_gt

## Protocol

In [ ]:
protocol = {}
protocol['Point Fact'] = {}
protocol['Point Fact']['ad_idx'] = 1
protocol['Point Fact']['score_idx'] = 2
protocol['Point Fact']['type'] = ['Point Fact']
protocol['Cluster Identification'] = {}
protocol['Cluster Identification']['ad_idx'] = 1
protocol['Cluster Identification']['score_idx'] = 2
protocol['Cluster Identification']['type'] = ['Cluster Identification']
protocol['Cluster Facet'] = {}
protocol['Cluster Facet']['ad_idx'] = 1
protocol['Cluster Facet']['score_idx'] = 2
protocol['Cluster Facet']['type'] = ['Cluster Facet']
protocol['Cluster Aggregate'] = {}
protocol['Cluster Aggregate']['ad_idx'] = 1
protocol['Cluster Aggregate']['score_idx'] = 2
protocol['Cluster Aggregate']['type'] = ['MODE', 'MIN', 'MAX', 'AVG']

In [ ]:
AvgP = {}

## Create Dictionary

In [ ]:
for qtype in protocol.keys():
    AvgP[qtype] = {}

    sub_data = Submissions[qtype]
    gt_data = Ground_Truth[qtype]

    try:
        del submissions
    except:
        pass
    for sub in sub_data:
        if sub['type'] not in protocol[qtype]['type']:
            print sub['type']
            print "TROUBLE"
            print qtype
        submissions = clean_sub(sub['answer'], protocol[qtype]['ad_idx'], protocol[qtype]['score_idx'])
        q_id = sub['question_id']
        if '-' in q_id:
            q_id = q_id.split('-')[0]
        gt = gt_data[q_id]
        AvgP[qtype][q_id] = AvgPrecision(gt, submissions, protocol[qtype]['ad_idx'], protocol[qtype]['score_idx'])

## Remove Question 94 from Aggregate

In [ ]:
try:
    del AvgP['Cluster Aggregate']['94']
except:
    pass

## Reformat Output

In [ ]:
NewEval = {}

for qtype in AvgP.keys():
    NewEval[qtype] = {}
    for quest in AvgP[qtype].keys():
        NewEval[qtype][quest] = {}
        NewEval[qtype][quest]['avg_precision'] = AvgP[qtype][quest]

## Calculate Overall Aggregate Metric and Reformat Output

In [ ]:
for quest in Submissions['Cluster Aggregate']:
    q_id = quest['question_id'].split('-')[0]
    # Initiate metric at 0
    metric = 0
    # Skip submission for question 94
    if q_id != '94':
        if len(quest['answer']) > 0:
            # Some may be empty strings
            if len(quest['answer'][0]) > 0:
                # Some answers have strings
                if ',' in quest['answer'][0]:
                    sub_answer = float(quest['answer'][0].split(',')[0])
                else:
                    sub_answer = float(quest['answer'][0])
                metric = AggMetric(agg_answ_gt[q_id]['agg_answer'], sub_answer, agg_answ_gt[q_id]['feature'])
        NewEval['Cluster Aggregate'][q_id]['agg_metric'] = metric

## Calculate Point Fact Extraction Metric

## Save Avg Precision Dict

In [ ]:
save_file = extractions +'.json'
with open(save_file, 'w') as outfile:
    json.dump(NewEval, outfile, indent=2)

In [ ]:
#for quest in Submissions['Cluster Aggregate']:
#    print quest['answer'][0]